In [1]:
DATA_PATH = '../../data/CRTS2/'
RAW_DATA_PATH = DATA_PATH + 'raw/'

In [2]:
import pandas as pd
import numpy as np

Load raw .out data

In [3]:
df_out = pd.read_csv(RAW_DATA_PATH + 'all.out')
print(df_out.shape)

(4280, 5)


Pre-process .phot data

In [4]:
# Load Raw Data
df_phot = pd.read_csv(RAW_DATA_PATH + 'all.phot')
print(df_phot.shape)

(482538, 6)


In [5]:
# Remove non CSS Data
query = df_phot.CatalinaID.astype(str).str.startswith('1')
df_phot = df_phot[query]
print(df_phot.shape)

(388440, 6)


In [6]:
# Add Transient ID from .out to .phot
df_phot['TransientID'] = 0
df_phot['TransientID'].astype(np.int64)
for catID in df_phot.CatalinaID.unique():
    row = df_out[df_out.CatalinaID == catID]
    transID = row.TransientID.iloc[0]
    df_phot.loc[df_phot.CatalinaID == catID, 'TransientID'] = transID

In [7]:
df_phot.TransientID.unique().shape

(2300,)

In [8]:
# Remove unnecessary fields, and reorder columns
df_phot = df_phot[['TransientID', 'Mag', 'Magerr', 'MJD']]
print(df_phot.shape)

(388440, 4)


Pre-process .orph data

In [9]:
# Load raw data
df_orph = pd.read_csv(RAW_DATA_PATH + 'all.orph')
print(df_orph.shape)

(70515, 12)


In [10]:
df_orph.TransientID.unique().shape

(3092,)

In [11]:
# Remove non CSS Data
query = df_orph.FieldID.astype(str).str.startswith('1')
df_orph = df_orph[query]
print(df_orph.shape)

(63034, 12)


In [12]:
# Remove unnecessary fields, and reorder columns
df_orph = df_orph[['TransientID', 'Mag', 'Magerr', 'MJD']]
print(df_orph.shape)

(63034, 4)


Merge .phot and .orph dataframes

In [13]:
df = df_phot.copy().append(df_orph, ignore_index=True)

In [14]:
df.head()

,TransientID,Mag,Magerr,MJD
0,1409030010044114444,18.8765,0.166417,53766.089871
1,1409030010044114444,20.0519,0.281733,53990.458866
2,1409030010044114444,20.2199,0.295764,53996.286004
3,1409030010044114444,21.1192,0.495390,54385.205789
4,1409030010044114444,19.3289,0.195002,54355.282285


In [15]:
outdir = DATA_PATH
filename = 'transient_lightcurves.pickle' 
outpath = outdir + filename
df.to_pickle(outpath)